   # **Spectrogram Analysis**
   A spectrogram is a visual representation of the spectrum of frequencies in a sound signal as they vary with time. It is a powerful tool for analyzing the frequency content of audio signals, identifying patterns, and understanding the temporal evolution of sound.

In [ ]:
%pip install librosa numpy matplotlib

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import logging

from IPython.display import Audio, display

In [ ]:
# Adjust sys.path to include the root directory
logging.info("Adjusting sys.path to include the root directory")
root_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
logging.info(f"sys.path adjusted: {sys.path}")

In [ ]:
# Local Imports & Parameters
logging.info("Local configurations and parameters imported")
from config.config import audio_config, output_config
from config.parameters import *
from config.logging import setup_logging

# Plot configurations
from config.matplotlib_plots import configure_plot
logging.info("Local configurations and parameters imported")

In [ ]:
# Set up logging for this notebook
logging.info("Setting up logging for the notebook")
notebook_path = os.path.join(os.getcwd(), 'spectogram.ipynb')
setup_logging(notebook_path)
logging.info("Logging set up complete")

In [ ]:
def load_audio(file_key):
    """
    Load an audio file.

    Args:
        file_key (str): Key to identify the audio file in AUDIO_FILES.

    Returns:
        tuple: Audio time series and sampling rate.
    """
    audio_file = audio_config.get_audio_file(file_key)
    if not audio_file:
        logging.error(f"Audio file key '{file_key}' not found.")
        raise ValueError(f"Audio file key '{file_key}' not found.")
    
    try:
        y, sr = librosa.load(audio_file)
        logging.info(f"Loaded audio file: {audio_file}")
        return y, sr
    
    except Exception as e:
        logging.error(f"Failed to load audio file: {e}")
        raise

In [ ]:
def compute_spectrogram(y, sr, n_fft=DEFAULT_N_FFT, hop_length=DEFAULT_HOP_LENGTH):
    """
    Compute the spectrogram of the audio signal.

    Args:
        y (np.ndarray): Audio time series.
        sr (int): Sampling rate.
        n_fft (int, optional): Number of FFT components. Defaults to DEFAULT_N_FFT.
        hop_length (int, optional): Number of samples between successive frames. Defaults to DEFAULT_HOP_LENGTH.

    Returns:
        np.ndarray: Spectrogram.
    """
    D = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    logging.info("Computed spectrogram.")
    return D

In [ ]:
def plot_and_save_spectrogram(D, sr, hop_length, output_path):
    """
    Plot and save the spectrogram.

    Args:
        D (np.ndarray): Spectrogram.
        sr (int): Sampling rate.
        hop_length (int): Number of samples between successive frames.
        output_path (str): Path to save the spectrogram plot.
    """
    fig, ax = plt.subplots(figsize=FIGURE_SIZE)
    
    # Set the background color for the figure and axes
    fig.patch.set_facecolor(BACKGROUND_COLOR)
    ax.set_facecolor(BACKGROUND_COLOR)
    
    img = librosa.display.specshow(
        librosa.amplitude_to_db(D, ref=np.max),
        sr=sr,
        hop_length=hop_length,
        x_axis='time',
        y_axis='log',
        ax=ax
    )
    cbar = fig.colorbar(img, format='%+2.0f dB')
    cbar.ax.set_facecolor(BACKGROUND_COLOR)
    
    # Set color for the color bar labels
    cbar.ax.yaxis.set_tick_params(color=SPINE_COLOR)
    plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color=SPINE_COLOR)
    
    # Configure the plot using the existing function
    configure_plot(ax, title='Spectrogram', subtitle='Frequency Domain Representation')
    
    plt.tight_layout()
    plt.savefig(output_path, facecolor=BACKGROUND_COLOR)
    plt.show()
    
    logging.info(f"Spectrogram plot saved to: {output_path}")

In [ ]:
# Load Audio File
y, sr = load_audio(AUDIO_FILE_SAX_A3)

In [ ]:
# Compute Spectrogram
D = compute_spectrogram(y, sr)

In [ ]:
# Define Output Path
output_dir = output_config.get_output_directory("frequency_domain")
output_path = os.path.join(output_dir, 'Spectrogram.png')

# Plot and Save Spectrogram
plot_and_save_spectrogram(D, sr, DEFAULT_HOP_LENGTH, output_path)